In [2]:
# Dependencies
import plotly.graph_objs as go
from shapely.geometry import shape
import plotly.express as px
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
import json

### Migration flows by Region

Information about the data can be fount here: https://www.nomisweb.co.uk/sources/census_2021_mig

In [3]:
# Create df for migration into England and Wales
mig_rgn_in_df = pd.read_csv("./Resources/MIG004EW_RGN_IN.csv")
# print(mig_rgn_in_df.head())

# Create df for emigration from England and Wales
mig_rgn_out_df = pd.read_csv("./Resources/MIG004EW_RGN_OUT.csv")
# mig_rgn_out_df.head()

In [4]:
# Count for regions in flow
mig_rgn_in = mig_rgn_in_df.groupby(['Migration Region (inflow) (6 categories) code', 'Migration Region (inflow) (6 categories) label'])['Count'].sum().reset_index()
mig_rgn_in = mig_rgn_in.sort_values(by='Count', ascending=False)

#Save df to csv file
mig_rgn_in.to_csv("./Resources/local_authority_in.csv", index=False)


# Count local regions out flow
mig_rgn_out = mig_rgn_out_df.groupby(['Migration Region (outflow) (3 categories) code', 'Migration Region (outflow) (3 categories) label'])['Count'].sum().reset_index()
mig_rgn_out = mig_rgn_out.sort_values(by='Count', ascending=False)


#Save df to csv file
mig_rgn_out.to_csv("./Resources/local_authority_out.csv", index=False)

print(mig_rgn_out)

   Migration Region (outflow) (3 categories) code  \
0                                              -8   
1                                               1   
2                                               2   

     Migration Region (outflow) (3 categories) label     Count  
0                                     Does not apply  57800607  
1  Outflow: moved out of the area, but within the...   1091937  
2  Outflow: moved out of the associated area, but...     97709  


### Merge of Net Migration information by region

In [5]:
# Count for regions in flow
mig_rgn_in = mig_rgn_in_df.groupby(['Regions code','Regions label'])['Count'].sum().reset_index()
mig_rgn_in = mig_rgn_in.sort_values(by='Count', ascending=False)
mig_rgn_in

mig_rgn_out = mig_rgn_out_df.groupby(['Migrant region one year ago code', 'Migrant region one year ago label'])['Count'].sum().reset_index()
mig_rgn_out = mig_rgn_out.sort_values(by='Count', ascending=False)
# mig_rgn_out

# Merge DataFrames on the common column
region_df = pd.merge(mig_rgn_in, mig_rgn_out, left_on='Regions code', right_on='Migrant region one year ago code', how='left')

# Fill NaN values with 0
region_df[['Count_x', 'Count_y']] = region_df[['Count_x', 'Count_y']].fillna(0)

# Calculate net migration
region_df['Net Migration'] = region_df['Count_x'] - region_df['Count_y']

# Select relevant columns
region_net_migration = region_df[['Regions code', 'Regions label', 'Net Migration']]

#Save df to csv file
region_net_migration.to_csv("./Resources/region_net_migration.csv", index=False)


In [6]:
# Load the CSV data
csv_data = pd.read_csv("./Resources/region_net_migration.csv")

# Load the GeoDataFrame from the GeoJSON file
geojson_data = pd.read_csv("./Resources/European_Electoral_Regions_(December_2017)_UK_BGC.csv")
# geojson_data
# Merge the CSV data with the GeoJSON data based on 'Lower tier local authorities code'
merged_rgn = pd.merge(geojson_data, csv_data, left_on='eer17nm', right_on='Regions label', how='right')

merged_rgn = merged_rgn.dropna().reset_index()
merged_rgn

,index,OBJECTID,eer17cd,eer17nm,bng_e,bng_n,long,lat,GlobalID,SHAPE_Length,SHAPE_Area,Regions code,Regions label,Net Migration
0,0,8.0,E15000008,South East,470062.0,172924.0,-1.00000,50.000000,{AE3DD173-7004-4554-BFDF-0300B6C8F062},2.344929e+06,1.908870e+10,E12000008,South East,8246470
1,1,7.0,E15000007,London,517516.0,178392.0,-0.30864,51.492271,{2676FB0F-D3E2-4555-B48F-6F9854C45B46},4.098115e+05,1.573490e+09,E12000007,London,7520262
2,2,2.0,E15000002,North West,350015.0,506280.0,-2.77237,54.449451,{E2D72183-0640-4B11-A22B-D12F79D6BE7C},2.004775e+06,1.416366e+10,E12000002,North West,6688909
3,4,5.0,E15000005,West Midlands,386294.0,295477.0,-2.20358,52.556969,{8440C830-94C2-4741-9413-41D496CD9773},9.214201e+05,1.300378e+10,E12000005,West Midlands,5368354
4,5,9.0,E15000009,South West,285015.0,102567.0,-3.63343,50.811192,{C23B888A-3941-42B4-B603-FE128F7D4A95},3.189895e+06,2.385184e+10,E12000009,South West,5067886
5,6,3.0,E15000003,Yorkshire and The Humber,446903.0,448736.0,-1.28712,53.932640,{9E42A0C4-1C72-40D2-B8C3-3F5AC0233AF0},1.359201e+06,1.540941e+10,E12000003,Yorkshire and The Humber,4918006
6,7,4.0,E15000004,East Midlands,477660.0,322635.0,-0.84967,52.795719,{99C36B73-7560-43E9-9A26-60C5CB27E149},1.298382e+06,1.564311e+10,E12000004,East Midlands,4373113
7,8,12.0,W08000001,Wales,263406.0,242881.0,-3.99416,52.067410,{1A9286D3-C892-4019-9638-22C827E56233},2.895629e+06,2.078215e+10,W92000004,Wales,2818571
8,9,1.0,E15000001,North East,417313.0,600358.0,-1.72890,55.297031,{6EE094C7-B281-4906-AEEA-BFA06E836AB8},9.836890e+05,8.592564e+09,E12000001,North East,2388495


### Net Migration per Region

In [7]:
# Convert the 'geometry' column to GeoJSON format
merged_rgn['geometry'] = merged_rgn['geometry'].apply(shape)

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(merged_rgn, geometry='geometry')

# Create a GeoJSON representation of the GeoDataFrame
geojson_data = gdf.to_crs(epsg=4326).__geo_interface__  # Ensure the GeoJSON is in WGS 84 (EPSG:4326)

# Creating the net flow choropleth map with plotly.express
fig = px.choropleth_mapbox(
    gdf,
    geojson=geojson_data,
    locations=gdf.index,
    color='Net Migration',
    color_continuous_scale="BuPu",
    mapbox_style="carto-positron",
    center={
        "lat": gdf['lat'].mean(),
        "lon": gdf['long'].mean(),
    },
    zoom=5.3,
    opacity=0.7,
    labels={'Net Migration': 'Net Immigration:'},
    custom_data=['UTLA Code', 'UTLA Label', 'Net Migration'],
)

# Update hovertemplate to control the content of the tooltip
fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>%{customdata[1]}<br>Net Immigration: %{customdata[2]:,.0f}')

# Customize the layout to show a comma as a thousand separator in color bar tick labels
fig.update_layout(coloraxis_colorbar=dict(
    tickformat=','
))

# Adjust image size 
fig.update_layout(
    width=800,
    height=800
)

# Show or save the figure
fig.show()
# or fig.write_html("your_file.html")

KeyError: 'geometry'